In [1]:
'''
Editor: @Hyunhomo
Data source: https://www.phmsociety.org/events/conference/phm/18/data-challenge

This implementation can be used for convert industrial raw data log to run-to-fail data
DataFrame contains RUL(time-to-failure) with cycles
'''

## Import libraries in python
import os
import pandas as pd
import padasip as pa
# pa.filters.FilterRLS(n)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import importlib
from scipy.stats import randint, expon, uniform

import sklearn as sk
from sklearn import svm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn import preprocessing
from sklearn import pipeline
from sklearn.metrics import mean_squared_error
from math import sqrt

In [58]:
# How many rows to read 
num_cycles = 3000000

## Load and assign training data which includes all sensor and other measurements(attributes)
df_01_M02_DC_train = pd.read_csv('Data/01_M02_DC_train.csv', nrows = num_cycles)
pd.set_option('display.max_rows', 2000)

# df_01_M02_DC_train.head(2000)
df_01_M02_DC_train

'''
recipe
recipe_step
IONGAUGEPRESSURE : pressure reading for the main process chamber when under vacuum
ETCHSUPPRESSORCURRENT : ion current impacting the suppressor grid plate
FLOWCOOLFLOWRATE : rate of flow of helium through the flowcool circuit, controlled by mass flow controller
FLOWCOOLPRESSURE : resulting helium pressure in the flowcool circuit


'''

print (df_01_M02_DC_train['recipe'].unique())
print (df_01_M02_DC_train['recipe_step'].unique())


df_01_M02_DC_train_reduced = df_01_M02_DC_train[['time','recipe', 'recipe_step', 'IONGAUGEPRESSURE', 
                                                'ETCHSUPPRESSORCURRENT', 'FLOWCOOLFLOWRATE', 'FLOWCOOLPRESSURE']]


df_01_M02_DC_train_reduced.to_csv('Data/01_M02_DC_train_reduced.csv', index=False)
print ("csv file saved")

[67 68 66 73 69 71 72 70 74 76 77 75]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51  0 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73]
csv file saved


In [60]:
df_01_M02_DC_train_reduced = pd.read_csv('Data/01_M02_DC_train_reduced.csv')
df_01_M02_DC_train_reduced

,time,recipe,recipe_step,IONGAUGEPRESSURE,ETCHSUPPRESSORCURRENT,FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE
0,3283834,67,1,-0.079428,-0.939043,-2.842907,-2.332807
1,3283838,67,1,-0.079419,-0.939043,-2.842907,-2.332807
2,3283842,67,1,-0.079409,-0.939043,-2.841850,-2.332807
3,3283846,67,1,-0.079420,-0.939043,-2.841850,-2.332807
4,3283850,67,1,-0.079430,-0.939043,-2.841850,-2.332807
...,...,...,...,...,...,...,...
2999995,25633642,67,33,0.014749,1.503257,-0.282087,-0.193548
2999996,25633646,67,33,0.014749,1.451337,-0.282087,-0.194201
2999997,25633650,67,33,0.014749,1.411106,-0.307455,-0.197424
2999998,25633654,67,33,0.014749,1.391854,-0.280125,-0.198709


In [94]:
# df_01_M02_DC_train = pd.read_csv('Data/ttf/01_M02_DC_train.csv', nrows = num_cycles, header=None,  index_col=False)
df_01_M02_DC_ttf = pd.read_csv('Data/ttf/01_M02_DC_train.csv', nrows = num_cycles)
pd.set_option('display.max_rows', 2000)
# print (df_01_M02_DC_ttf)
# df_01_M02_DC_ttf.head(2000)
df_01_M02_DC_ttf[int(614786/5+36132/5+500)-500:int(614786/5+36132/5+500)+500]


time_FPDBL_failure = df_01_M02_DC_ttf['time'].loc[df_01_M02_DC_ttf['TTF_FlowCool Pressure Dropped Below Limit'] == 0]
# print (time_FPDBL_failure)

FPDBL_idx = time_FPDBL_failure.index.values
FPDBL_time = time_FPDBL_failure.values
print (FPDBL_idx)
print (FPDBL_time)



# del_idx = np.where(np.in1d(FPDBL_idx, [146084,146565,822821,822821,822933,823355,2693250,2893945]))[0]

# FPDBL_idx_ed = np.delete(FPDBL_idx, del_idx)
# FPDBL_time_ed = np.delete(FPDBL_time, del_idx)
# print (FPDBL_idx_ed)
# print (FPDBL_time_ed)


[ 131033  144072  146084  146565  816430  822332  822821  822933  823355
 2203555 2645858 2693250 2827173 2893945 2915376]
[ 3907068  3962388  3971424  3973348  8072052  8123030  8124986  8125434
  8127122 18108796 22575152 22793870 24521052 25002546 25136976]
[ 131033  144072  816430  822332 2203555 2645858 2827173 2915376]
[ 3907068  3962388  8072052  8123030 18108796 22575152 24521052 25136976]


In [77]:
## Load fault data if needed
df_01_M02_train_fault_data = pd.read_csv('Data/01_M02_train_fault_data.csv', nrows = num_cycles)
# pd.set_option('display.max_rows', 1000)
# df_01_M02_train_fault_data


time_FPDBL_fault = df_01_M02_train_fault_data['time'].loc[df_01_M02_train_fault_data['fault_name'] == 'FlowCool Pressure Dropped Below Limit']
# print (time_FPDBL_fault)

In [118]:
##
df_train_ttf = pd.concat([df_01_M02_DC_train_reduced, df_01_M02_DC_ttf], axis=1)
# df_train = df_01_M02_DC_train_reduced
# Slicing training data for each failure cycle
# 
df_train_ttf_list = []
prev_idx = 0
# for f_idx in FPDBL_idx:
for cycle, f_idx in enumerate(FPDBL_idx, start=1):
    print (cycle)
    print (prev_idx , f_idx)
    cycle_array = np.zeros(f_idx-prev_idx+1, dtype=int)+cycle
    temp_df = df_train_ttf[prev_idx:f_idx+1] 
    temp_df['failure_cycle'] = cycle_array
    df_train_ttf_list.append(temp_df)
    prev_idx = f_idx +1


df_train_ttf_list[1]


1
0 131033
2
131034 144072
3
144073 146084
4
146085 146565
5
146566 816430
6
816431 822332
7
822333 822821
8
822822 822933
9
822934 823355
10
823356 2203555
11
2203556 2645858
12
2645859 2693250
13
2693251 2827173
14
2827174 2893945
15
2893946 2915376


/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,time,recipe,recipe_step,IONGAUGEPRESSURE,ETCHSUPPRESSORCURRENT,FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE,time,TTF_FlowCool Pressure Dropped Below Limit,TTF_Flowcool Pressure Too High Check Flowcool Pump,TTF_Flowcool leak,failure_cycle
131034,3907072,67,43,-0.022398,0.532254,0.567441,0.221736,3907072,7998.0,8523232.0,9276.0,2
131035,3907076,67,43,-0.019429,0.532935,0.571069,0.221736,3907076,7994.0,8523228.0,9272.0,2
131036,3907080,67,43,-0.019429,0.532254,0.571069,0.221736,3907080,7990.0,8523224.0,9268.0,2
131037,3907084,67,43,-0.019429,0.536724,0.571069,0.221736,3907084,7986.0,8523220.0,9264.0,2
131038,3907088,67,43,-0.019429,0.527439,0.568199,0.221736,3907088,7982.0,8523216.0,9260.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
144068,3962372,67,49,0.011108,0.590015,0.571069,0.207580,3962372,16.0,8467932.0,8528.0,2
144069,3962376,67,49,0.011108,0.582796,0.564421,0.205010,3962376,12.0,8467928.0,8524.0,2
144070,3962380,67,49,0.011108,0.585200,0.564421,0.207580,3962380,8.0,8467924.0,8520.0,2
144071,3962384,67,49,0.011108,0.580385,0.564421,0.207580,3962384,4.0,8467920.0,8516.0,2


In [119]:
df_train_ttf_merged = pd.concat(df_train_ttf_list)
df_train_ttf_merged

#Save to csv file

,time,recipe,recipe_step,IONGAUGEPRESSURE,ETCHSUPPRESSORCURRENT,FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE,time,TTF_FlowCool Pressure Dropped Below Limit,TTF_Flowcool Pressure Too High Check Flowcool Pump,TTF_Flowcool leak,failure_cycle
0,3283834,67,1,-0.079428,-0.939043,-2.842907,-2.332807,3283834,623234.0,9146470.0,632514.0,1
1,3283838,67,1,-0.079419,-0.939043,-2.842907,-2.332807,3283838,623230.0,9146466.0,632510.0,1
2,3283842,67,1,-0.079409,-0.939043,-2.841850,-2.332807,3283842,623226.0,9146462.0,632506.0,1
3,3283846,67,1,-0.079420,-0.939043,-2.841850,-2.332807,3283846,623222.0,9146458.0,632502.0,1
4,3283850,67,1,-0.079430,-0.939043,-2.841850,-2.332807,3283850,623218.0,9146454.0,632498.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2915372,25136960,67,23,-0.022719,0.801476,0.565479,0.185082,25136960,16.0,6288598.0,10202482.0,15
2915373,25136964,67,23,-0.022719,0.804920,0.565479,0.185082,25136964,12.0,6288594.0,10202478.0,15
2915374,25136968,67,23,-0.022719,0.806979,0.565479,0.182512,25136968,8.0,6288590.0,10202474.0,15
2915375,25136972,67,23,-0.022719,0.798039,0.570147,0.182512,25136972,4.0,6288586.0,10202470.0,15
